In [1]:
from datetime import date, datetime

def get_column_datatype(cell):
    """
    Gets the Redshift datatype for a column in the csv
    
    type cell: can be one of the following types:
        int, decimal, float, 
    """ 
    #subfunction checking if a cell is a float or not
    def _isfloat(s):
        try:
            float(s)
            return True
        except ValueError:
            return False
    
    #subfunction checking if a cell is a bool or not
    #NOTE: if a cell has a bool datatype then it must be
    #      initially recorded with capital 'T' for 'True' or capital 'F' for 'False'
    def _isbool(s):
        if s=='True' or s=='False':
            return True
        else:
            return False
    
    #subfunction checking if a cell is a date or not
    #NOTE: this subfunction only accounts for two different date formats
    #      from the CSV. 
    #      Format One: 8 digits, where the first 4
    #          digits represent year, then next 2 digits represent month, and
    #          final two digits represent day
    #      Format Two: 2 digits for day of the month followed by
    #          string of 3 letters representing month (first 3 letters of month name)
    #          followed by 4 digits for year
    def _isdate(s):
        #Format One check
        if s.isdigit() and len(s)==8:
            potential_year = int(s[:4].lstrip('0'))
            potential_month = int(s[4:6].lstrip('0'))
            potential_day = int(s[6:].lstrip('0'))
            if potential_year in range(0,2016) and potential_month in range(1,12) \
                and potential_day in range(1,31):
                    return True             
        #Format Two check
        if len(s)==9:
            potential_year2 = int(s[5:].lstrip('0'))
            potential_day2 = int(s[:2].lstrip('0'))
            #checking if 3 letters in the middle of the string represent a valid month
            if s[2:5] in ('JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', \
                'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC') and potential_year2 in range(0,2016) \
                and potential_day2 in range(1,31):
                    return True
        #if neither Format One of Format Two
        return False
    
    #strip of all white space before anf after the string
    cell = cell.strip(' ')
    if _isdate(cell):
        return 'date'
    elif cell.isdigit():
        return 'integer'
    elif _isfloat(cell):
        return 'real'
    elif _isbool(cell):
        return 'bool'
    else:
        return 'varchar(256)'

In [2]:
#INPUT : SOME CSV FILE
#OUTPUT: SQL QUERY TO CREATE THE TABLE ITSELF IN REDSHIFT
        # AND SQL QUERY TO POPULATE TABLE

import csv
    
def get_query_from_csv(some_csv):
    with open(some_csv, 'r') as f:
        reader = csv.reader(f)
        csv_rows_list = list(reader)
        
    headers = csv_rows_list[0] #your headers for each of the columns
    columns_datatypes_list = [get_column_datatype(column_cell) for column_cell in csv_rows_list[1]]  
    
    column_headers_and_types = \
        ', '.join([headers[i] + " " + columns_datatypes_list[i] for i in range(0, len(headers))])
    
    table_name, csv_ext = some_csv.split('.')
    final_query = "CREATE TABLE IF NOT EXISTS " + table_name + " (" + \
        column_headers_and_types + ")"
    
    print(csv_rows_list)
    
    return final_query

In [3]:
a_data = get_query_from_csv("sample.csv")

[['head1', 'head2', 'head3', 'head4'], ['1', ' 2', ' 3.0', ' 19310810'], ['4', ' 5', ' 6.0', ' 10MAR2010'], ['7', ' 8', ' 9.0', ' 09MAR2010'], ['10', ' 11', ' 12.0', ' 09MAR2010'], ['13', ' 14', ' 15.0', ' 09MAR2010']]


In [4]:
a_data

'CREATE TABLE IF NOT EXISTS sample (head1 integer, head2 integer, head3 real, head4 date)'

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("sample.csv")

In [7]:
df.columns.values

array(['head1', 'head2', 'head3', 'head4'], dtype=object)

In [8]:
df

,head1,head2,head3,head4
0,1,2,3,19310810
1,4,5,6,10MAR2010
2,7,8,9,09MAR2010
3,10,11,12,09MAR2010
4,13,14,15,09MAR2010


In [9]:
def funcA(x):
    return x*x

In [10]:
df['head1'] = df[['head1']].apply(funcA)

In [21]:
df.dtypes

head1      int64
head2      int64
head3    float64
head4     object
dtype: object

In [20]:
for thing in df:
    print(df[thing][0])

1
2
3.0
 19310810


In [23]:
def _isdate(s):
    #Format One check
    if s.isdigit() and len(s)==8:
        potential_year = int(s[:4].lstrip('0'))
        potential_month = int(s[4:6].lstrip('0'))
        potential_day = int(s[6:].lstrip('0'))
        if potential_year in range(0,2016) and potential_month in range(1,12) \
            and potential_day in range(1,31):
                return True             
    #Format Two check
    if len(s)==9:
        potential_year2 = int(s[5:].lstrip('0'))
        potential_day2 = int(s[:2].lstrip('0'))
        #checking if 3 letters in the middle of the string represent a valid month
        if s[2:5] in ('JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', \
            'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC') and potential_year2 in range(0,2016) \
            and potential_day2 in range(1,31):
                return True
    #if neither Format One of Format Two
    return False


In [52]:
df = pd.read_csv('/Users/jlee/Downloads/COTA_Health_Claims_19.txt', delimiter='|')

/Users/jlee/.pyenv/versions/3.4.3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,10,13,14,22,26,27,29,30,33,43,61,64,67,68,69,71,72,73,74,75,76,77,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
df.head(100)

,CLM_SRC_ID,CLM_SRC_SYS_CD,CLM_LN_SRC_ID,MAIN,MAJ_PROD,HZN_ID,SUBS_ID,PAT_FNM,PAT_LNM,PAT_REL,...,IN_OUT_IND,IND_REND_TIN,REND_FIRST_NAME,REND_LAST_NAME,hznkey,ICD_VER_CD,mask_it,efdt,trmdt,CANCER_DIAG_FDOS
0,10152002890,QMX,1,055272,HMO,3HZN18936880,NaN,NaN,NaN,SELF,...,IN,NaN,NaN,NaN,NaN,9,Y,05/20/2006,05/19/2014,11JAN2012
1,10186000418,QMX,1,055272,HMO,3HZN18936880,NaN,NaN,NaN,SELF,...,IN,NaN,NaN,NaN,NaN,9,Y,05/20/2006,05/19/2014,11JAN2012
2,10265001223,QMX,1,055272,HMO,3HZN18936880,NaN,NaN,NaN,SELF,...,IN,NaN,NaN,NaN,NaN,9,Y,05/20/2006,05/19/2014,11JAN2012
3,10266001485,QMX,1,055272,HMO,3HZN18936880,NaN,NaN,NaN,SELF,...,IN,NaN,NaN,NaN,NaN,9,Y,05/20/2006,05/19/2014,11JAN2012
4,10280001287,QMX,1,055272,HMO,3HZN18936880,NaN,NaN,NaN,SELF,...,IN,NaN,NaN,NaN,NaN,9,Y,05/20/2006,05/19/2014,11JAN2012
5,10295001426,QMX,1,055272,HMO,3HZN18936880,NaN,NaN,NaN,SELF,...,IN,NaN,NaN,NaN,NaN,9,Y,05/20/2006,05/19/2014,11JAN2012
6,10298002426,QMX,1,055272,HMO,3HZN18936880,NaN,NaN,NaN,SELF,...,IN,NaN,NaN,NaN,NaN,9,Y,05/20/2006,05/19/2014,11JAN2012
7,10300001308,QMX,1,055272,HMO,3HZN18936880,NaN,NaN,NaN,SELF,...,IN,NaN,NaN,NaN,NaN,9,Y,05/20/2006,05/19/2014,11JAN2012
8,10314A02090,QMX,1,00003Y,HMO,3HZN01254460,NaN,MOLLY,SHEPHERD,SELF,...,IN,NaN,NaN,NaN,3HZN01254460MOL-16621,9,N,01/01/2011,12/31/2013,08MAR2012
9,10314A02090,QMX,2,00003Y,HMO,3HZN01254460,NaN,MOLLY,SHEPHERD,SELF,...,IN,NaN,NaN,NaN,3HZN01254460MOL-16621,9,N,01/01/2011,12/31/2013,08MAR2012


In [59]:
df.head(100)['trmdt'].apply(_isdate)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
      ...  
70    False
71    False
72    False
73    False
74    False
75    False
76    False
77    False
78    False
79    False
80    False
81    False
82    False
83    False
84    False
85    False
86    False
87    False
88    False
89    False
90    False
91    False
92    False
93    False
94    False
95    False
96    False
97    False
98    False
99    False
Name: trmdt, dtype: bool

In [57]:
df.head(100)['CANCER_DIAG_FDOS'].apply(_isdate)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
      ... 
70    True
71    True
72    True
73    True
74    True
75    True
76    True
77    True
78    True
79    True
80    True
81    True
82    True
83    True
84    True
85    True
86    True
87    True
88    True
89    True
90    True
91    True
92    True
93    True
94    True
95    True
96    True
97    True
98    True
99    True
Name: CANCER_DIAG_FDOS, dtype: bool

In [51]:
df[['head4']]

,head4
0,19310810
1,10MAR2010
2,09MAR2010
3,09MAR2010
4,09MAR2010


In [12]:
df

,head1,head2,head3,head4
0,1,2,3,19310810
1,16,5,6,10MAR2010
2,49,8,9,09MAR2010
3,100,11,12,09MAR2010
4,169,14,15,09MAR2010


True

In [13]:
a_data

'CREATE TABLE IF NOT EXISTS sample (head1 integer, head2 integer, head3 real, head4 date)'

In [14]:
a_data

'CREATE TABLE IF NOT EXISTS sample (head1 integer, head2 integer, head3 real, head4 date)'